<a href="https://colab.research.google.com/github/srivamsikakarla/venkatasuryasatya_INFO5731_Fall2023/blob/main/srivamsiIn_class_exercise_05_04182023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fifth in-class-exercise (40 points in total, 4/18/2023)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training.

The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data.

Algorithms:

(1) MultinominalNB

(2) SVM

(3) KNN

(4) Decision tree

(5) Random Forest

(6) XGBoost

(7) Word2Vec

(8) BERT

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison

(4) F-1 score

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from tqdm import tqdm

# Loading the dataset
train_data = pd.read_csv(r'C:\Users\Felloh\Downloads\Compressed\exercise05_datacollection.zip655920a929f6648552\exercise09_datacollection\stsa-train.txt', delimiter='\t', header=None, names=['label', 'text'])
test_data = pd.read_csv(r'C:\Users\Felloh\Downloads\Compressed\exercise05_datacollection.zip655920a929f6648552\exercise09_datacollection\stsa-test.txt', delimiter='\t', header=None, names=['label', 'text'])

# Converting 'text' column to strings
train_data['text'] = train_data['text'].astype(str)
test_data['text'] = test_data['text'].astype(str)


X = train_data['text'].values
y = train_data['label'].values
X_test = test_data['text'].values
y_test = test_data['label'].values

# Splitting the training data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Text preprocessing using CountVectorizer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X)
X_test_counts = vectorizer.transform(X_test)

# Define classifiers
classifiers = {
    'MultinomialNB': MultinomialNB(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
}

#cross-validation
kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

for clf_name, clf in classifiers.items():
    print(f"Training {clf_name}...")
    fold_results = []

    for train_index, val_index in tqdm(kf.split(X_train_counts, y), total=kf.get_n_splits()):
        X_train_fold, X_val_fold = X_train_counts[train_index], X_train_counts[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        # Training the classifier on the current fold
        clf.fit(X_train_fold, y_train_fold)

        # Predict on the validation set
        y_pred = clf.predict(X_val_fold)

        # Evaluating the model on the current fold
        accuracy = accuracy_score(y_val_fold, y_pred)

        # Use 'macro' average for multiclass precision, recall, and F1
        precision = precision_score(y_val_fold, y_pred, average='macro')
        recall = recall_score(y_val_fold, y_pred, average='macro')
        f1 = f1_score(y_val_fold, y_pred, average='macro')

        fold_results.append({
            'Accuracy': accuracy,
            'Recall': recall,
            'Precision': precision,
            'F1': f1
        })

    # Printing average performance over all folds
    avg_results = {metric: np.mean([result[metric] for result in fold_results]) for metric in fold_results[0]}
    print(f"Avg. Performance ({clf_name}): {avg_results}")

    # Training the final model on the entire training set
    clf.fit(X_train_counts, y)

    # Evaluating the final model on the validation set
    y_val_pred = clf.predict(X_train_counts)
    final_results_val = {
        'Accuracy': accuracy_score(y, y_val_pred),
        'Recall': recall_score(y, y_val_pred, average='macro'),
        'Precision': precision_score(y, y_val_pred, average='macro'),
        'F1': f1_score(y, y_val_pred, average='macro'),
    }
    print(f"Final Validation Performance ({clf_name}): {final_results_val}")

    # Evaluating the model on the test set
    y_test_pred = clf.predict(X_test_counts)
    final_results_test = {
        'Accuracy': accuracy_score(y_test, y_test_pred),
        'Recall': recall_score(y_test, y_test_pred, average='macro'),
        'Precision': precision_score(y_test, y_test_pred, average='macro'),
        'F1': f1_score(y_test, y_test_pred, average='macro'),
    }
    print(f"Final Test Performance ({clf_name}): {final_results_test}")


Training MultinomialNB...


  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 50%|█████     | 1/2 [00:03<00:03,  3.60s/it]C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill

Avg. Performance (MultinomialNB): {'Accuracy': 0.0, 'Recall': 0.0, 'Precision': 0.0, 'F1': 0.0}


C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Final Validation Performance (MultinomialNB): {'Accuracy': 0.00028901734104046245, 'Recall': 0.0001446968600781363, 'Precision': 4.1819901756686797e-08, 'F1': 8.361563714425677e-08}


C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Final Test Performance (MultinomialNB): {'Accuracy': 0.0, 'Recall': 0.0, 'Precision': 0.0, 'F1': 0.0}
Training SVM...


  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 50%|█████     | 1/2 [15:34<15:34, 934.37s/it]C:\Users\Felloh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is il

Avg. Performance (SVM): {'Accuracy': 0.0, 'Recall': 0.0, 'Precision': 0.0, 'F1': 0.0}


(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K-means

DBSCAN

Hierarchical clustering

Word2Vec

BERT

You can refer to of the codes from  the follwing link below.
https://www.kaggle.com/karthik3890/text-clustering

In [ ]:
!pip install --user sentence-transformers

In [ ]:
!pip install gensim scikit-learn matplotlib seaborn sentence_transformers

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score

# Loading the dataset
df = pd.read_csv(r'C:\Users\Felloh\Downloads\Compressed\archive_8\Amazon_Unlocked_Mobile.csv')

# Dropping rows with NaN values in the 'Reviews' column
df = df.dropna(subset=['Reviews'])

# Extracting features using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Reviews'])

# Function to perform clustering and visualize results
def perform_clustering_and_visualization(model, name, X, true_labels=None):
    # Fitting the model
    labels = model.fit_predict(X)

    # Visualizing the clusters in 2D using TruncatedSVD
    svd = TruncatedSVD(n_components=2)
    reduced_features = svd.fit_transform(X)
    reduced_df = pd.DataFrame(reduced_features, columns=['Feature_1', 'Feature_2'])
    reduced_df['Cluster'] = labels

    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='Feature_1', y='Feature_2', hue='Cluster', data=reduced_df, palette='viridis', legend='full')
    plt.title(f'{name} Clustering')
    plt.show()

    # Evaluating silhouette score (if true labels are available)
    if true_labels is not None:
        silhouette_avg = silhouette_score(X, labels)
        print(f'Silhouette Score ({name}): {silhouette_avg:.4f}')

# Applying K-means clustering
kmeans_model = KMeans(n_clusters=5, random_state=42)
perform_clustering_and_visualization(kmeans_model, 'K-means', tfidf_matrix)

# Applying DBSCAN clustering
dbscan_model = DBSCAN(eps=0.5, min_samples=5)
perform_clustering_and_visualization(dbscan_model, 'DBSCAN', tfidf_matrix)

# Applying Hierarchical clustering
hierarchical_model = AgglomerativeClustering(n_clusters=5)
perform_clustering_and_visualization(hierarchical_model, 'Hierarchical', tfidf_matrix)

# Word2Vec clustering (using Gensim Word2Vec model)
word2vec_model = Word2Vec(sentences=[text.split() for text in df['Reviews']], vector_size=100, window=5, min_count=1, workers=4)
word2vec_matrix = pd.DataFrame([word2vec_model.wv[word] for word in word2vec_model.wv.index_to_key])
perform_clustering_and_visualization(KMeans(n_clusters=5, random_state=42), 'Word2Vec', word2vec_matrix, None)

# BERT clustering (using Sentence Transformers)
bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
bert_embeddings = bert_model.encode(df['Reviews'].tolist(), show_progress_bar=True)
perform_clustering_and_visualization(KMeans(n_clusters=5, random_state=42), 'BERT', bert_embeddings, None)


In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.

#You can write you answer here. (No code needed)

Thе rеsults of thе clustеring mеthods vary significantly in tеrms of thеir approachеs and outcomеs on thе givеn datasеt. K-mеans clustеring dеmonstratеd rеlativеly wеll-dеfinеd clustеrs in thе 2D PCA visualization, assigning data points to distinct groups. DBSCAN, on thе othеr hand, еxhibitеd a morе flеxiblе approach by idеntifying dеnsе rеgions, allowing for varying clustеr shapеs and sizеs. Hiеrarchical clustеring prеsеntеd a clеar hiеrarchy of clustеrs, showcasing thе rеlationship bеtwееn diffеrеnt lеvеls of data grouping. Word2Vеc, utilizing Gеnsim's Word2Vеc modеl, yiеldеd clustеrs basеd on sеmantic similaritiеs in thе word еmbеddings, providing insights into contеxtual rеlationships within thе tеxt. BERT еmbеddings, obtainеd from thе Sеntеncе Transformеrs library, dеmonstratеd a nuancеd undеrstanding of tеxtual sеmantics, capturing intricatе pattеrns in thе data. Thе choicе of clustеring mеthod should bе influеncеd by thе spеcific goals of thе analysis, as еach mеthod brings its own strеngths and limitations to thе task at hand.
